In [4]:
import pandas as pd
from Bio import SeqIO, AlignIO, Seq
import numpy as np
from collections import Counter
from helper import *
import python_cipres.client as CipresClient

### Extract fasta sequences for the autotrophic rubiscos
Before running this code we need to generate a file called `autotrophic_rubisco_70p.csv` by selecting in ITOL clades that belong to Rubisco types 1,2,2/3,3a,3c,3-like,3b and IV. We replaced sapces in the sequence ID to `_` to match the sequence ID in the fasta files. We take only sequences which are not type IV or type III-b as autotrophic.

In [5]:
uclust_data = pd.read_csv('../output/01_70p_tree/uclust_all_0.7.csv')
uclust_data['cut Target'] = uclust_data.Target.apply(lambda x: x.split(' ')[0])
#auto_id = [x.replace('\n','') for x in open('../output/01_70p_tree/autotrophic_rubisco_70p.txt').readlines() if x != '\n']
rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')

rubisco_with_type = uclust_data.merge(rubisco_types, left_on='cut Target', right_on='ID')
rubisco_with_type[rubisco_with_type.type != 'IV'].groupby('type')['Query'].count().sum()

37897

In [7]:
uclust_data = pd.read_csv('../output/01_70p_tree/uclust_all_0.7.csv')
uclust_data['cut Target'] = uclust_data.Target.apply(lambda x: x.split(' ')[0])
#auto_id = [x.replace('\n','') for x in open('../output/01_70p_tree/autotrophic_rubisco_70p.txt').readlines() if x != '\n']
rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')
autotrophic_70p = rubisco_types[~rubisco_types['type'].isin(['IV','IIIb'])]
auto_rubisco = uclust_data.merge(autotrophic_70p, left_on='cut Target', right_on='ID')
autotrophic_rubisco  = auto_rubisco['Query'].values

auto_seq = []
for record in SeqIO.parse('../output/00_100p_tree/uclust_all_1.faa', "fasta"):
    if record.description in autotrophic_rubisco:
        auto_seq.append(record)
#!mkdir -p ../output/02_90p_autotrophic_rubisco_tree
with open(r"../output/02_90p_autotrophic_rubisco_tree/autotrophic_rubisco_seq.faa", "w") as output_handle:
    SeqIO.write(auto_seq, output_handle, "fasta")

In [10]:
auto_rubisco.to_csv('../output/02_90p_autotrophic_rubisco_tree/uclust_all_0.7_with_type.csv')
auto_rubisco.groupby('type')['Query'].nunique()

type
I              36218
II               475
II/III            95
III-like         118
IIIa              93
IIIc              50
IV-outgroup        1
unknown           81
Name: Query, dtype: int64

### Cluster sequences using uclust

In [4]:
!../bin/usearch11.0.667_i86linux32 -cluster_fast ../output/02_90p_autotrophic_rubisco_tree/autotrophic_rubisco_seq.faa -id 0.9 -uc ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.uc

usearch v11.0.667_i86linux32, 4.0Gb RAM (16.3Gb total), 8 cores
(C) Copyright 2013-18 Robert C. Edgar, all rights reserved.
https://drive5.com/usearch

License: yinonmoise.baron@weizmann.ac.il

00:00 64Mb    100.0% Reading ../output/02_90p_autotrophic_rubisco_tree/autotrophic_rubisco_seq.faa
00:00 102Mb   100.0% DF
00:00 103Mb  37131 seqs, 37131 uniques, 37131 singletons (100.0%)
00:00 103Mb  Min size 1, median 1, max 1, avg 1.00
00:00 108Mb   100.0% DB
00:02 168Mb   100.0% 1022 clusters, max size 6960, avg 36.3
                                                           
      Seqs  37131 (37.1k)
  Clusters  1022
  Max size  6960
  Avg size  36.3
  Min size  1
Singletons  507, 1.4% of seqs, 49.6% of clusters
   Max mem  168Mb
      Time  2.00s
Throughput  18.6k seqs/sec.



### Take only cluster fasta files and create csv file

In [5]:
parse_uclust(infile='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.uc',
             fasta='../output/02_90p_autotrophic_rubisco_tree/autotrophic_rubisco_seq.faa',
             outfasta='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa',
             outfile='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.csv'
            )

### Remove outliers - sequences which less than 50% of them map to Rr sequence by blast

In [6]:
from Bio.Blast.Applications import NcbiblastpCommandline
output = NcbiblastpCommandline(query="../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa", subject="../data/Rr.faa", outfmt=5)()[0]
from Bio.Blast import NCBIXML
from io import StringIO

res = []
for x in NCBIXML.parse(StringIO(output)):
    if len(x.alignments) == 0:
        res.append(0)
    else:
        alignment = x.alignments[0]
        res.append(pd.DataFrame([[x.align_length,x.expect] for x in alignment.hsps]).sort_values(by=1).loc[0,0]/x.query_length)

seq = [x for x in SeqIO.parse('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa',format='fasta')]

res_df = pd.Series(res,index=[x.id for x in seq])

In [7]:
seqs = [x for x in seq if x.id in res_df[res_df >= 0.5].index]

with open(r"../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.faa", "w") as output_handle:
    SeqIO.write(seqs, output_handle, "fasta")
    
seqs = [x for x in seq if x.id in res_df[res_df < 0.5].index]
with open(r"../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_outliers.faa", "w") as output_handle:
    SeqIO.write(seqs, output_handle, "fasta")

In [8]:
auto_rub_90p = pd.read_csv('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.csv')
no_outliers = [x.id for x in seq if x.id in res_df[res_df >= 0.5].index]
auto_rub_90p_no_outliers = auto_rub_90p[auto_rub_90p['Target'].apply(lambda x: x.split(' ')[0]).isin(no_outliers)]
auto_rub_90p_no_outliers.to_csv('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv')
auto_rub90 = auto_rubisco.merge(auto_rub_90p_no_outliers,left_on='Query',right_on='Query', suffixes=('_70','_90'))

auto_rub90.to_csv('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers_with_type.csv')
auto_rub90.groupby('type')['Target_90'].nunique()

type
I              550
II             174
II/III          52
III-like        66
IIIa            66
IIIc            36
IV-outgroup      1
unknown         64
Name: Target_90, dtype: int64

## Create multiple sequence alignment

In [9]:
#!../bin/mafft-linux64/mafft.bat ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.faa > ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.aln
!../bin/mafft-linux64/mafft.bat ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.faa > ../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.aln

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
rescale = 1
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
 1001 / 1009
done.

Constructing a UPGMA tree (efffree=0) ... 
 1000 / 1009
done.

Progressive alignment 1/2... 
STEP   901 / 1008 
Reallocating..done. *alloclen = 2425

Reallocating..done. *alloclen = 3762
STEP  1001 / 1008  h
done.

Making a distance matrix from msa.. 
 1000 / 1009
done.

Constructing a UPGMA tree (efffree=1) ... 
 1000 / 1009
done.

Progressive alignment 2/2... 
STEP   901 / 1008 
Reallocating..done. *alloclen = 2510
STEP  1001 / 1008  h
Reallocating..done. *alloclen = 3925

done.

disttbfast (aa) Version 7.427
alg=A, model=BLOSUM62, 1.53, -0.00, -0.00, noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.


## Clean MSA to contain only positions with more than 5% coverage (based on Jaffe et al. 2018)

In [10]:
clean_aln(infile='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9.aln',
          outfile='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_trimmed.aln')

In [11]:
cip = CipresClient.Client(appname='RO',
                    appID='rubisco_phylogeny-49F87B124F3D429FBE12F95E4254DDEA',
                    baseUrl='https://cipresrest.sdsc.edu/cipresrest/v1',
                    username='yinonbaron',
                    password='Mchcav11~')


In [12]:
job = cip.submitJob(vParams={'toolId': 'RAXMLHPC8_REST_XSEDE',
                       'datatype_': 'protein',
                       'runtime_': '160',
                       'select_analysis_': 'fa',
                       'choose_bootstrap_': 'x',
                       'printbrlength_': '1'},
              inputParams={'infile_': '../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_trimmed.aln'},
              metadata={'statusEmail':'true'})

In [10]:
job = cip.listJobs()[-1]

In [14]:
if cip.getJobStatus(jobHandle=job.jobHandle).isDone():
    !mkdir -p ../output/02_90p_autotrophic_rubisco_tree/RaxML/
    job.downloadResults('../output/02_90p_autotrophic_rubisco_tree/RaxML/')
else:
    print('Job ' + job.jobHandle + ' not finished')

In [6]:
d = pd.read_csv('../output/00_100p_tree/uclust_all_1_rubisco_types.csv')
d2 = pd.read_csv('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv')
d3 = d2.merge(d, left_on='Query', right_on='ID',how='left')
d3.type.unique()

array([nan, 'I', 'IIIlike', 'II', 'II/III', 'IIIc', 'unknown', 'IIIa',
       'IIIb'], dtype=object)

In [15]:
add_type(type_file='../output/00_100p_tree/uclust_all_1_rubisco_types.csv',
         seq_file='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv',
         outfile='../output/02_90p_autotrophic_rubisco_tree/type_legend.txt')

In [4]:
add_kinetic(kinetic_file='../output/00_100p_tree/uclust_all_1_kinetic_data.csv',
         synth_file='../output/00_100p_tree/synth_data.csv',
         seq_file='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv',
         outfile='../output/02_90p_autotrophic_rubisco_tree/kinetic_legend.txt')

In [2]:
add_kinetic_on_label(kinetic_file='../output/00_100p_tree/uclust_all_1_kinetic_data.csv',
         synth_file='../output/00_100p_tree/synth_data.csv',
         seq_file='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv',
         outfile='../output/02_90p_autotrophic_rubisco_tree/kinetic_legend_label.txt')


['gi|604722319|dbj|BAO57366.1|,label,node,#B2BABB,1,normal\n'
 'gi|604722319|dbj|BAO57366.1|,label,node,#B2BABB,1,normal\n'
 'gi|604722319|dbj|BAO57366.1|,label,node,#B2BABB,1,normal\n' ...
 'gi|576253|pdb|1RBL|A,label,node,#B2BABB,1,normal\n'
 'gi|157678845|dbj|BAF80663.1|,label,node,#B2BABB,1,normal\n'
 'RBC2_50,label,node,#B2BABB,1,normal\n']


In [23]:
k = pd.read_csv('../output/00_100p_tree/uclust_all_1_kinetic_data.csv')
k

,kinetic_ID
0,RBC2_50 gi|499819577|ref|WP_011500311.1|
1,gi|499561661|ref|WP_011242444.1| MULTISPECIES:...
2,gi|499709017|ref|WP_011389751.1| ribulose-bisp...
3,gi|502736740|ref|WP_012971724.1| ribulose bisp...
4,"gi|356472757|gb|AET10441.1| ribulose-1,5-bisph..."
5,"gi|11467200|ref|NP_043033.1| ribulose-1,5-bisp..."
6,"gi|14017580|ref|NP_114267.1| ribulose-1,5-bisp..."
7,"gi|11497536|ref|NP_054944.1| ribulose-1,5-bisp..."
8,"gi|1248646408|gb|ATG33856.1| ribulose-1,5-bisp..."
9,gi|157678845|dbj|BAF80663.1| ribulose 1.5-bisp...


In [1]:
uclust = rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')
uclust70 = pd.read_csv('../output/01_70p_tree/uclust_all_0.7.csv')
rubisco_types = pd.read_csv('../output/01_70p_tree/rubisco_types_70p.csv')
#uclust[uclust.Query.str.contains('499819577')]
#uclust70[uclust70.Query.str.contains('499819577')]


In [14]:
uclust = pd.read_csv('../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers_with_type.csv')
t = uclust[uclust['type'].isin(['II','II/III'])]
syn = pd.read_csv('../output/00_100p_tree/synth_data.csv',header=None)
mdata_syn = t.merge(syn,left_on='Query',right_on=0)
mdata_syn

,Unnamed: 0,Type_70,Cluster_70,Size_70,%Id_70,Strand_70,Qlo_70,Tlo_70,Alignment_70,Query,...,Type_90,Cluster_90,Size_90,%Id_90,Strand_90,Qlo_90,Tlo_90,Alignment_90,Target_90,0
0,0,H,1,486,74.2,.,0,0,2I472M14D,RBCSeed_18 gi|563352309|gb|AHB41464.1|,...,C,2,2,*,*,*,*,*,RBCSeed_18 gi|563352309|gb|AHB41464.1|,RBCSeed_18 gi|563352309|gb|AHB41464.1|
1,1,H,1,479,85.4,.,0,0,474M5D,RBC4_4 gi|504865161|ref|WP_015052263.1|,...,C,17,1,*,*,*,*,*,RBC4_4 gi|504865161|ref|WP_015052263.1|,RBC4_4 gi|504865161|ref|WP_015052263.1|
2,2,H,1,477,83.3,.,0,0,I473M4D,RBC4_6 gi|503663498|ref|WP_013897574.1|,...,C,18,1,*,*,*,*,*,RBC4_6 gi|503663498|ref|WP_013897574.1|,RBC4_6 gi|503663498|ref|WP_013897574.1|
3,7,H,1,484,77.8,.,0,0,D474M9D,RBCSeed_31 RIFCSPLOWO2_02_FULL_OP11_38_8_rifcs...,...,C,265,2,*,*,*,*,*,RBCSeed_31 RIFCSPLOWO2_02_FULL_OP11_38_8_rifcs...,RBCSeed_31 RIFCSPLOWO2_02_FULL_OP11_38_8_rifcs...
4,8,H,1,487,74.7,.,0,0,2D474M11D,RBCSeed_35 TARA_138.SAMEA2623390.450.0.22-3_15...,...,C,269,1,*,*,*,*,*,RBCSeed_35 TARA_138.SAMEA2623390.450.0.22-3_15...,RBCSeed_35 TARA_138.SAMEA2623390.450.0.22-3_15...
5,9,H,1,475,79.3,.,0,0,D474M,RBCSeed_5 gi|1004818558|gb|KYC44365.1|,...,C,270,2,*,*,*,*,*,RBCSeed_5 gi|1004818558|gb|KYC44365.1|,RBCSeed_5 gi|1004818558|gb|KYC44365.1|
6,10,H,1,474,89.5,.,0,0,474M,RBC4_7 gi|1223654907|ref|WP_091690387.1|,...,C,276,3,*,*,*,*,*,RBC4_7 gi|1223654907|ref|WP_091690387.1|,RBC4_7 gi|1223654907|ref|WP_091690387.1|
7,18,H,1,479,86.5,.,0,0,474M5D,RBC3_4 WP_023846593.1,...,C,429,4,*,*,*,*,*,RBC3_4 WP_023846593.1,RBC3_4 WP_023846593.1
8,19,H,1,483,79.3,.,0,0,D474M8D,RBC4_2 gi|1004827596|gb|KYC52615.1|,...,H,270,483,91.8,.,0,0,475M8D,RBCSeed_5 gi|1004818558|gb|KYC44365.1|,RBC4_2 gi|1004827596|gb|KYC52615.1|
9,20,H,1,479,86.7,.,0,0,474M5D,RBC4_3 gi|1223901358|ref|WP_091933438.1|,...,H,429,479,92.7,.,0,0,479M,RBC3_4 WP_023846593.1,RBC4_3 gi|1223901358|ref|WP_091933438.1|
